In [114]:
import pickle
import pandas as pd
from math import log
import re
import nltk
nltk.download(['punkt','stopwords','wordnet','words'])
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cmejia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cmejia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cmejia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\cmejia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


In [2]:
loaded_model = pickle.load(open('estructuraDatos.sav', 'rb'))
idexFiles = loaded_model['idexFiles']
vectorizer = loaded_model['vectorizer']
matrix = loaded_model['matriz']

In [3]:
# Encontrar los documentos que contiene una palabra en particular
def encontrarDoc(palabra):
    col = vectorizer.vocabulary_[palabra]
    matx = matrix[:,col]
    indx = matx.nonzero()[0]
    lista =indx.tolist() 
    dfresult = pd.DataFrame()
    for i in range(len(lista)):
        auxres= pd.DataFrame({'NombreArchivo': idexFiles[lista[i]], 'Frecuencia': [matx.data[i]]})
        dfresult = pd.concat([dfresult, auxres])
    dfresult.sort_values('Frecuencia',ascending = False,inplace = True)
    return dfresult

In [84]:
def indice_invertido(dic):
    inv = {}
    N = matrix.shape[0]
    for k, v in vectorizer.vocabulary_.items():
        inv.setdefault(k, {})
        #Los documentos que contienen la palabra v
        matx = matrix[:,v]
        #Indicador de los documentos que contienen la palabra
        indx = matx.nonzero()[0]
        lista =indx.tolist()
        #Calculo del IDF, lista contiene todos los documentos que contienen la palabra
        inv[k]['IDF'] = log(N/(len(lista)+1))
        docs = {}
        for i in range(len(lista)):
            keys = docs.setdefault(idexFiles[lista[i]], [])
            #Frecuencia de la palabra V en el documento lista[i]
            keys.append(matx.data[i])
            #Las palabras que contiene el documento lista[i]
            matx2 = matrix[lista[i],:]
            #La frecuencia de cada palabra, que sumada el vector da el total de palabras en el documento
            keys.append(matx2.data.sum())
        inv[k]['Documentos'] = docs
    return inv

In [85]:
ind_inv = indice_invertido(vectorizer.vocabulary_)

In [ ]:
stopWords = stopwords.words('english')
def queryClean(texto):
    #Pasar todo a minisculas
    texto = texto.lower()
    texto =re.sub('(á|à|ä)','a',texto) # Reemplazar a acentuada
    texto =re.sub('(é|è|ë)','e',texto) # Reemplazar e acentuada
    texto =re.sub('(í|ì|ï)','i',texto) # Reemplazar i acentuada
    texto =re.sub('(ó|ò|ö)','o',texto) # Reemplazar o acentuada
    texto =re.sub('(ú|ù|ü)','u',texto) # Reemplazar u acentuada
    texto =re.sub('[^a-zA-Z]',' ',texto) # Eliminar caracteres que no sean: letra, número o vocales acentuadas
    texto =re.sub(' +',' ',texto) # Eliminar espacios en blanco
    #Tokenizar
    tokens = texto.split()
    tokens = [w for w in tokens if (len(w)>1)&(w.isalpha())&(w not in stopWords)]
    #Lemma
    word_net_lemmatizar = WordNetLemmatizer()
    tokens = [word_net_lemmatizar.lemmatize(w, pos = "v") for w in tokens]

    #Stemmer
    ps = PorterStemmer() 
    tokens = [ps.stem(w) for w in tokens]

    return tokens

In [109]:
def queryTF(word):
    respuesta = sorted(ind_inv[word]['Documentos'].items(), key = lambda kv:(kv[1], kv[0]),reverse=True)
    return respuesta

In [105]:
def queryTFDL(word):
    aux = ind_inv[word]['Documentos']
    auxdic = {}
    for k,v in aux.items():
        keys = auxdic.setdefault(k, [])
        keys.append(v[0]/v[1])
    respuesta = sorted(auxdic.items(), key = lambda kv:(kv[1], kv[0]),reverse=True)
    return respuesta

In [135]:
verquery = queryClean('intelligent')
respuesta1 = queryTF(verquery[0])
respuesta2 = queryTFDL(verquery[0])

In [138]:
verquery

['intellig']

In [139]:
encontrarDoc(verquery[0])

,NombreArchivo,Frecuencia
0,1212.1710.txt,81
0,1307.0449.txt,69
0,1411.0440.txt,46
0,1411.5878.txt,28
0,1110.2053.txt,27
0,1508.03891.txt,26
0,1506.08544.txt,26
0,1509.02709.txt,20
0,1503.01239.txt,20
0,1312.0049.txt,18


In [136]:
respuesta1

[('1212.1710.txt', [81, 17264]),
 ('1307.0449.txt', [69, 36584]),
 ('1411.0440.txt', [46, 14830]),
 ('1411.5878.txt', [28, 11491]),
 ('1110.2053.txt', [27, 29769]),
 ('1508.03891.txt', [26, 17838]),
 ('1506.08544.txt', [26, 9644]),
 ('1509.02709.txt', [20, 10129]),
 ('1503.01239.txt', [20, 5748]),
 ('1312.0049.txt', [18, 10267]),
 ('1412.5902.txt', [15, 8932]),
 ('1403.1080.txt', [15, 5877]),
 ('1301.2959.txt', [15, 2827]),
 ('1510.07380.txt', [14, 9645]),
 ('1301.2158.txt', [12, 4725]),
 ('1309.0671.txt', [12, 3002]),
 ('1402.6208.txt', [12, 2355]),
 ('1506.02930.txt', [12, 2147]),
 ('1509.04634.txt', [10, 6822]),
 ('1101.5460.txt', [10, 6119]),
 ('1502.05767.txt', [9, 9797]),
 ('1511.01258.txt', [9, 6387]),
 ('1511.06860.txt', [9, 4009]),
 ('1401.6312.txt', [8, 8818]),
 ('1307.2783.txt', [8, 6900]),
 ('1505.04026.txt', [8, 5246]),
 ('1306.1167.txt', [8, 3263]),
 ('1304.3944.txt', [8, 2941]),
 ('1505.05451.txt', [8, 2618]),
 ('0911.1546.txt', [8, 2406]),
 ('1510.03840.txt', [8, 1739])

In [137]:
respuesta2

[('1506.02930.txt', [0.0055891942244993015]),
 ('1301.2959.txt', [0.005305978068623983]),
 ('1402.6208.txt', [0.005095541401273885]),
 ('1212.1710.txt', [0.004691844300278036]),
 ('1510.03840.txt', [0.004600345025876941]),
 ('1309.0671.txt', [0.003997335109926716]),
 ('1509.03915.txt', [0.0035149384885764497]),
 ('1503.01239.txt', [0.003479471120389701]),
 ('0911.1546.txt', [0.0033250207813798837]),
 ('1411.0440.txt', [0.0031018206338503033]),
 ('1505.05451.txt', [0.0030557677616501145]),
 ('1304.3944.txt', [0.0027201632097925877]),
 ('1506.08544.txt', [0.0026959767731231854]),
 ('1403.4539.txt', [0.002607561929595828]),
 ('1403.1080.txt', [0.002552322613578356]),
 ('1301.2158.txt', [0.0025396825396825397]),
 ('1306.1167.txt', [0.002451731535396874]),
 ('1411.5878.txt', [0.002436689583152032]),
 ('1511.06860.txt', [0.0022449488650536295]),
 ('1511.08280.txt', [0.0020898641588296763]),
 ('1407.0756.txt', [0.002040122407344441]),
 ('1410.6516.txt', [0.002008032128514056]),
 ('1510.07391.